<h2>Desarrollo del Desafio 2 - Machine Learning


<h3>Grupo 7

<li>Laura Jazmín Chao
<li>Juan Nicolás Capistrano
<li>Betiana Srur
<li>Ignacio Mendieta
<li>Marecelo Carrizo

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [1]:
import numpy as np
import pandas as pd

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Libreria para analisis ML
from sklearn import datasets, linear_model

In [2]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
64252,64252,sell,house,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|,Argentina,Bs.As. G.B.A. Zona Norte,3427753.0,NaN,NaN,...,390.0,3589.743590,3589.743590,NaN,NaN,4000.0,http://www.properati.com.ar/1aak5_venta_casa_t...,"Exelente casa sobre el río en Albanueva, acces...",Imperdible casa sobre el rio en barrio cerrado,https://thumbs4.properati.com/8/m-rpgt5WlR39j2...
80477,80477,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9441158,-60.6662684",-32.944116,...,53.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1aww2_venta_depart...,Unidad monoambiente. Cocina. Estar/comedor. Ba...,Departamento - Macrocentro,https://thumbs4.properati.com/0/mj3jg93VwY9gqq...
85848,85848,sell,apartment,Olivos,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430310.0,NaN,NaN,...,157.0,4170.854271,5286.624204,NaN,NaN,NaN,http://www.properati.com.ar/1b3lm_venta_depart...,Preventa ENTREGA INMEDIATA. Cocheras incluida ...,Departamento con vista el río 5 amb,https://thumbs4.properati.com/4/yIqWyL7D4MbyTj...
69986,69986,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,"-34.6428675967,-58.4376599743",-34.642868,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1aixd_venta_depart...,AVISO LEGAL: Las descripciones arquitectónicas...,Departamento en P.Chacabuco,https://thumbs4.properati.com/5/yyMiu8BHQI9KXC...
92136,92136,sell,apartment,Colón,|Argentina|Entre Ríos|Colón|,Argentina,Entre Ríos,3435351.0,NaN,NaN,...,40.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1bcgr_venta_depart...,"DPTOS DESDE EL POZO! A pocas cuadras del río,...","Centro de Colón, en zona muy alta y cerca del ...",https://thumbs4.properati.com/8/0VZxft7lPuNvsU...


In [3]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Limpieza de Datos.

In [4]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
# Observamos esos registros
data.loc[currency_OTHER_CURRENCY_mask, :]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [5]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data.shape[0]
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data.shape[0]}')

Cantidad de registros original - registro actuales: 3


In [6]:
# Creamos la lista de columnas a filtrar
drop_columns = ['properati_url', 'Unnamed: 0', 'operation','image_thumbnail','expenses','lat-lon','geonames_id','lat', 'lon']

# Creamos una variable nueva para guardar los datos (el cambio se realiza inplace=False)
data_filtered = data.drop(drop_columns, axis=1)
data_filtered.head(5)

,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


**Esto no es necesario ahora ¿?**

In [7]:
# Creamos un diccionario con la nueva agrupación
provincia={'Capital Federal':'CABA','Bs.As. G.B.A. Zona Norte': 'Bs. As. G.B.A.', 'Bs.As. G.B.A. Zona Sur':'Bs. As. G.B.A.', 'Córdoba':'Córdoba','Santa Fe':'Santa Fe','Buenos Aires Costa Atlántica':'Buenos Aires', 'Bs.As. G.B.A. Zona Oeste':'Bs. As. G.B.A.', 'Buenos Aires Interior':'Buenos Aires','Río Negro':'Resto del país','Neuquén':'Resto del país', 'Mendoza':'Resto del país', 'Tucumán':'Resto del país', 'Corrientes':'Resto del país', 'Misiones':'Resto del país', 'Entre Ríos':'Resto del país', 'Salta':'Resto del país', 'Chubut':'Resto del país', 'San Luis':'Resto del país', 'La Pampa':'Resto del país','Formosa':'Resto del país','Chaco': 'Resto del país', 'San Juan':'Resto del país','Tierra Del Fuego':'Resto del país', 'Catamarca':'Resto del país', 'Jujuy': 'Resto del país', 'Santa Cruz':'Resto del país', 'La Rioja':'Resto del país', 'Santiago Del Estero':'Resto del país'}

In [8]:
# Mapeamos las provincias con el diccionario y ponermos el grupo en una columna nueva
data_filtered['province_group'] = data_filtered.state_name.map(provincia)
display(data_filtered.sample(5))


,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title,province_group
16650,apartment,Quilmes,|Argentina|Bs.As. G.B.A. Zona Sur|Quilmes|,Argentina,Bs.As. G.B.A. Zona Sur,89500.0,USD,1579182.75,89500.0,NaN,36.0,NaN,2486.111111,NaN,1.0,Corredor Responsable: Marcelo Trujillo - CUCIC...,Departamento Bernal Centro a estrenar,Bs. As. G.B.A.
32373,apartment,Posadas,|Argentina|Misiones|Posadas|,Argentina,Misiones,40000.0,USD,705780.00,40000.0,NaN,30.0,NaN,1333.333333,NaN,NaN,Contacto: Karla Gonzalez - MLS ID # 420381004-...,Venta Monoambiente Posadas,Resto del país
104255,apartment,Rincón de Milberg,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rinc...,Argentina,Bs.As. G.B.A. Zona Norte,117000.0,USD,2064406.50,117000.0,84.0,79.0,1392.857143,1481.012658,NaN,NaN,Emprendimiento de pozo. Sobre la calle Rivadav...,Edificio Rivadavia - Departamento tipo duplex ...,Bs. As. G.B.A.
117068,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,83640.0,USD,1475785.98,83640.0,NaN,53.0,NaN,1578.113208,1.0,1.0,Departamento en Venta de 1 dorm. en Cordoba,CASA BLANCA CONSTRUCCIÓN EN ALTURA,Córdoba
97756,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,150000.0,USD,2646675.00,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Muy lindo departamento en venta en planta alta...,Departamento de 2 ambientes en La Quadra,Bs. As. G.B.A.


<h2>Tomamos solamente CABA para el analisis.

In [9]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data_filtered.state_name == 'Capital Federal' 
data_caba = data_filtered.loc[data_caba_mask, :]

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['province_group', 'country_name', 'state_name', 'place_with_parent_names','description', 'title'], inplace=True)

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Vemos la cantidad de datos solamente de CABA.
data_caba.shape

(32314, 12)

In [11]:
# Revisamos si hay barrios de caba mal indentificados "Ejemplo: Capital Federal"
pd.set_option('display.max_rows', 1000) # Para mostrar todas las filas.


In [12]:
# Dropeamos los de Capital Federal
capi_mask = data_caba.place_name == 'Capital Federal'
data_caba.drop(data_caba.loc[capi_mask, :].index, inplace=True)

In [13]:
data_caba.shape

(31019, 12)

In [14]:
data_caba.head(20)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0
16,PH,Mataderos,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0
19,apartment,Palermo,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0
21,apartment,Palermo,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,NaN,4.0


## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATICAS, Dummy (ejemplo: place_name, property_type).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Regresion Lineal Multiple
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



### Primero damos homogenidad a los datos

Vamos a realizar ciertas verificaciones de los datos.

In [15]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos el price_aprox_usd

mask_price_validation = data_caba.price != data_caba.price_aprox_usd

display(data_caba.loc[mask_price_validation, :].shape)

# Detectamos que hay 4138 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba.loc[mask_price_validation, :].sample(20))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados. Podemos hacerlo en base al calculo que realizamos en el Desafio 1 ??

(4138, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
113548,apartment,Puerto Madero,NaN,NaN,NaN,NaN,200.0,200.0,NaN,NaN,NaN,3.0
55353,apartment,San Telmo,1317900.0,ARS,1304152.10,73912.67,35.0,31.0,2111.790571,42512.903226,NaN,1.0
47624,house,Flores,NaN,NaN,NaN,NaN,380.0,300.0,NaN,NaN,NaN,NaN
66628,apartment,Belgrano,NaN,NaN,NaN,NaN,45.0,41.0,NaN,NaN,NaN,NaN
31241,house,Mataderos,NaN,NaN,NaN,NaN,267.0,NaN,NaN,NaN,NaN,5.0
118104,apartment,Villa Crespo,NaN,NaN,NaN,NaN,89.0,72.0,NaN,NaN,NaN,NaN
84226,apartment,Villa Urquiza,NaN,NaN,NaN,NaN,40.0,31.0,NaN,NaN,NaN,NaN
1722,house,Barracas,NaN,NaN,NaN,NaN,120.0,120.0,NaN,NaN,NaN,3.0
2846,store,San Nicolás,NaN,NaN,NaN,NaN,55.0,55.0,NaN,NaN,NaN,NaN
8343,apartment,Nuñez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Elimino primero los registros donde no tengo nada de informacion

umbral = 10
data_caba.dropna(axis = 0, thresh=umbral, inplace=True)

# Vuelvo a ver como quedaron los datos
mask_price_validation = data_caba.price != data_caba.price_aprox_usd
display(data_caba.loc[mask_price_validation, :].shape)
display(data_caba.loc[mask_price_validation, :].sample(20))

(1035, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
48032,apartment,San Telmo,1329600.0,ARS,1315730.07,74568.85,30.0,26.0,2485.628333,51138.461538,NaN,1.0
118585,apartment,Palermo,6489162.0,ARS,6421469.81,363936.06,164.0,123.0,2219.122317,52757.414634,NaN,NaN
112998,apartment,Colegiales,2465000.0,ARS,2439286.13,138246.26,46.0,7.0,3005.353478,352142.857143,3.0,NaN
76355,apartment,Belgrano,2256382.0,ARS,2232844.24,126546.19,39.0,34.0,3244.774103,66364.176471,NaN,NaN
118751,apartment,Villa Urquiza,1838450.0,ARS,1819271.99,103107.03,37.0,32.0,2786.676486,57451.562500,NaN,NaN
28072,apartment,Palermo,3311974.0,ARS,3277424.76,185747.67,NaN,67.0,NaN,49432.447761,103.0,2.0
42550,apartment,Villa Urquiza,1911834.0,ARS,1891890.57,107222.68,40.0,37.0,2680.567000,51671.189189,NaN,2.0
45767,apartment,Recoleta,1300977.0,ARS,1339035.63,75889.69,305.0,27.0,248.818656,48184.333333,3.0,NaN
76101,apartment,Colegiales,110000.0,ARS,108852.44,6169.20,47.0,47.0,131.259574,2340.425532,NaN,3.0
56228,apartment,Flores,2292593.0,ARS,2268677.58,128577.04,55.0,48.0,2337.764364,47762.354167,NaN,NaN


In [17]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan es porque es un departamente y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba


data_caba.surface_total_in_m2.fillna(data_caba.surface_covered_in_m2)

data_caba.head(50)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0
16,PH,Mataderos,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0
19,apartment,Palermo,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0
21,apartment,Palermo,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,NaN,4.0
29,apartment,Flores,75000.0,USD,1323337.50,75000.0,43.0,43.0,1744.186047,1744.186047,NaN,2.0


In [18]:
display(data_caba.floor.count())

# El piso del departamente es un dato muy incompleto por lo que se dropea

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['floor'], inplace=True)


2816

In [19]:
display(data_caba.rooms.count())

# En el caso de rooms, podemos tomar que el dato para los NaN es 0?? modifica esto el analisis para predecir? 

11904

In [20]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba.drop(columns=['price', 'currency'], inplace=True)

data_caba.sample(20)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
110439,apartment,Palermo,1552716.00,88000.00,24.0,24.0,3666.666667,3666.666667,1.0
17173,apartment,Constitución,1711516.50,97000.00,47.0,41.0,2063.829787,2365.853659,1.0
88846,apartment,Villa Urquiza,2303665.92,130560.00,64.0,46.0,2040.000000,2838.260870,NaN
42369,apartment,Constitución,688135.50,39000.00,26.0,26.0,1500.000000,1500.000000,1.0
87446,apartment,Villa Urquiza,1923250.50,109000.00,38.0,23.0,2868.421053,4739.130435,NaN
61537,store,Flores,8645805.00,490000.00,174.0,174.0,2816.091954,2816.091954,1.0
42281,apartment,Caballito,2082051.00,118000.00,52.0,46.0,2269.230769,2565.217391,2.0
82920,apartment,Belgrano,1936836.76,109770.00,41.0,35.0,2677.317073,3136.285714,NaN
55422,apartment,Belgrano,4411125.00,250000.00,91.0,91.0,2747.252747,2747.252747,4.0
12300,apartment,Boedo,3440677.50,195000.00,107.0,107.0,1822.429907,1822.429907,3.0
